In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import json
import re
from requests import get
import collections
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
rank_df = pd.read_csv("fifa_ranking.csv")

In [4]:
rank_groups = rank_df.groupby('rank_date')

In [5]:
wc2006_rankings = rank_groups.get_group('5/17/06')
wc2010_rankings = rank_groups.get_group('5/26/10')
wc2014_rankings = rank_groups.get_group('5/8/14')
wc2018_rankings = rank_groups.get_group('5/17/18')

In [6]:
tourney_df = pd.read_csv('WorldCupMatches.csv', encoding = 'ISO-8859-1')
tourney_df.head()

,Year,Datetime,Stage,Unnamed: 3,Unnamed: 4,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930,13 Jul 1930 - 15:00,Group 1,NaN,NaN,Pocitos,Montevideo,France,4,1,Mexico,,4444.0,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX
1,1930,13 Jul 1930 - 15:00,Group 4,NaN,NaN,Parque Central,Montevideo,USA,3,0,Belgium,,18346.0,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL
2,1930,14 Jul 1930 - 12:45,Group 2,NaN,NaN,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,24059.0,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA
3,1930,14 Jul 1930 - 14:50,Group 3,NaN,NaN,Pocitos,Montevideo,Romania,3,1,Peru,,2549.0,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER
4,1930,15 Jul 1930 - 16:00,Group 1,NaN,NaN,Parque Central,Montevideo,Argentina,1,0,France,,23409.0,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA


In [7]:
home_team_result = []
diff = tourney_df['Home Team Goals'] - tourney_df['Away Team Goals']
for i in range(len(diff)):
    if diff[i] > 0:
        home_team_result.append('W')
    elif diff[i] == 0:
        home_team_result.append('D')
    else:
        home_team_result.append('L')

In [8]:
for i in range(len(tourney_df)):
    if tourney_df['Win conditions'][i] != ' ':
        if tourney_df['Home Team Name'][i] in tourney_df['Win conditions'][i]:
            home_team_result[i] = 'W'
        else:
            home_team_result[i] = 'L'

tourney_df['Home Team Result'] = home_team_result

In [9]:
tourney_groups = tourney_df.groupby('Year')

In [10]:
wc_2006_tourney = tourney_groups.get_group(2006)
wc_2010_tourney = tourney_groups.get_group(2010)
wc_2014_tourney = tourney_groups.get_group(2014)

In [11]:
def rank_game (tourney, rank):
    rank['Home Team Initials'] = rank['country_abrv']
    rank['Away Team Initials'] = rank['country_abrv']
    rank_abv = rank[['rank', 'Home Team Initials', 'Away Team Initials']]
    merge_home = tourney.merge(rank_abv[['rank', 'Home Team Initials']], 
                               how = 'left', on = 'Home Team Initials')
    merge_home = merge_home.rename(columns={'rank': 'Home Rank'})
    merge_both = merge_home.merge(rank_abv[['rank', 'Away Team Initials']], 
                                  how = 'left', on = 'Away Team Initials') 
    merge_both = merge_both.rename(columns={'rank': 'Away Rank'})
    return merge_both

In [12]:
merged_2006 = rank_game(wc_2006_tourney, wc2006_rankings)
merged_2010 = rank_game(wc_2010_tourney, wc2010_rankings)
merged_2014 = rank_game(wc_2014_tourney, wc2014_rankings)

In [13]:
step1 = merged_2006.append(merged_2010, ignore_index = True)

In [14]:
merged_set = step1.append(merged_2014, ignore_index = True)

In [15]:
merged_set.head()

,Year,Datetime,Stage,Unnamed: 3,Unnamed: 4,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Home Team Result,Home Rank,Away Rank
0,2006,09 Jun 2006 - 18:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Munich",Munich,Germany,4,2,Costa Rica,,66000.0,2,1,ELIZONDO Horacio (ARG),GARCIA Dario (ARG),OTERO Rodolfo (ARG),97410100,97410001,GER,CRC,W,19,26
1,2006,09 Jun 2006 - 21:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,Poland,0,2,Ecuador,,52000.0,0,1,KAMIKAWA Toru (JPN),HIROSHIMA Yoshikazu (JPN),KIM Dae Young (KOR),97410100,97410002,POL,ECU,L,29,39
2,2006,10 Jun 2006 - 15:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Frankfurt",Frankfurt/Main,England,1,0,Paraguay,,48000.0,1,0,RODRIGUEZ Marco (MEX),CAMARGO Jose Luis (MEX),LEAL Leonel (CRC),97410100,97410003,ENG,PAR,W,10,33
3,2006,10 Jun 2006 - 18:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Dortmund",Dortmund,"rn"">Trinidad and Tobago",0,0,Sweden,,62959.0,0,0,MAIDIN Shamsul (SIN),PERMPANICH Prachya (THA),GHULOUM Eisa (UAE),97410100,97410004,TRI,SWE,D,47,16
4,2006,10 Jun 2006 - 21:00,Group C,NaN,NaN,"FIFA World Cup Stadium, Hamburg",Hamburg,Argentina,2,1,C¥Ë_te d'Ivoire,,49480.0,2,0,DE BLEECKERE Frank (BEL),HERMANS Peter (BEL),VROMANS Walter (BEL),97410100,97410005,ARG,CIV,W,9,32


In [16]:
for i in range(len(merged_set['Home Team Name'])):
    if '>' in merged_set['Home Team Name'][i] :
        merged_set['Home Team Name'][i] = merged_set['Home Team Name'][i][4:]
    if 'Ivoire' in merged_set['Home Team Name'][i]:
        merged_set['Home Team Name'][i] = 'Ivory Coast'
    if 'Iran' in merged_set['Home Team Name'][i]:
        merged_set['Home Team Name'][i] = 'Iran'

for i in range(len(merged_set['Away Team Name'])):
    if '>' in merged_set['Away Team Name'][i] :
        merged_set['Away Team Name'][i] = merged_set['Away Team Name'][i][4:]
    if 'Ivoire' in merged_set['Away Team Name'][i]:
        merged_set['Away Team Name'][i] = 'Ivory Coast'
    if 'Iran' in merged_set['Away Team Name'][i]:
        merged_set['Away Team Name'][i] = 'Iran'


In [17]:
country_factors = pd.read_csv("gdp_rain_temp.csv", encoding = 'ISO-8859-1')
country_factors.head()

,Unnamed: 0,country,temp_in_celcius,rain_mm_per_year,gdp(millions_dollars),population,gdp(dollars),gdp_dollar_per_person
0,0.0,Burkina Faso,28.25,748.0,13187.0,19751651.0,1.318700e+10,667.64
1,1.0,Mali,28.25,282.0,14998.0,19107706.0,1.499800e+10,784.92
2,2.0,Djibouti,28.00,220.0,2082.0,971408.0,2.082000e+09,2143.28
3,3.0,Senegal,27.85,686.0,16057.0,16294270.0,1.605700e+10,985.44
4,4.0,Mauritania,27.65,92.0,4985.0,4540068.0,4.985000e+09,1098.00


In [18]:
def country_game (country_facts, tourney):
    country_facts['Home Team Name'] = country_facts['country']
    merge_home = tourney.merge(country_facts[['Home Team Name', 'temp_in_celcius', 'rain_mm_per_year', 'gdp(millions_dollars)',
                                              'population', 'gdp_dollar_per_person']], 
                               how = 'left', on = 'Home Team Name')
    merge_home = merge_home.rename(columns={'temp_in_celcius': 'Home Temp', 
                                            'rain_mm_per_year': 'Home Rain',
                                            'gdp(millions_dollars)': 'Home GDP (millions)',
                                            'population': 'Home Population',
                                            'gdp_dollar_per_person': 'Home GDP per Person (dollars)'})
    country_facts['Away Team Name'] = country_facts['country']
    merge_both = merge_home.merge(country_facts[['Away Team Name', 'temp_in_celcius', 'rain_mm_per_year', 'gdp(millions_dollars)',
                                                 'population', 'gdp_dollar_per_person']], 
                                  how = 'left', on = 'Away Team Name')
    merge_both = merge_both.rename(columns={'temp_in_celcius': 'Away Temp', 
                                            'rain_mm_per_year': 'Away Rain',
                                            'gdp(millions_dollars)': 'Away GDP (millions)',
                                            'population': 'Away Population',
                                            'gdp_dollar_per_person': 'Away GDP per Person (dollars)'})
    return merge_both

In [19]:
merged_country_info = country_game(country_factors, merged_set)
merged_country_info = merged_country_info.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

merged_country_info.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Home Team Result,Home Rank,Away Rank,Home Temp,Home Rain,Home GDP (millions),Home Population,Home GDP per Person (dollars),Away Temp,Away Rain,Away GDP (millions),Away Population,Away GDP per Person (dollars)
0,2006,09 Jun 2006 - 18:00,Group A,"FIFA World Cup Stadium, Munich",Munich,Germany,4,2,Costa Rica,,66000.0,2,1,ELIZONDO Horacio (ARG),GARCIA Dario (ARG),OTERO Rodolfo (ARG),97410100,97410001,GER,CRC,W,19,26,8.50,700.0,3684816.0,82293457.0,44776.54,24.80,2926.0,58056.0,4953199.0,11720.91
1,2006,09 Jun 2006 - 21:00,Group A,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,Poland,0,2,Ecuador,,52000.0,0,1,KAMIKAWA Toru (JPN),HIROSHIMA Yoshikazu (JPN),KIM Dae Young (KOR),97410100,97410002,POL,ECU,L,29,39,7.85,600.0,524886.0,38104832.0,13774.79,21.85,2087.0,102311.0,16863425.0,6067.04
2,2006,10 Jun 2006 - 15:00,Group B,"FIFA World Cup Stadium, Frankfurt",Frankfurt/Main,England,1,0,Paraguay,,48000.0,1,0,RODRIGUEZ Marco (MEX),CAMARGO Jose Luis (MEX),LEAL Leonel (CRC),97410100,97410003,ENG,PAR,W,10,33,8.45,1220.0,2624529.0,66573504.0,39423.03,23.55,1130.0,29619.0,6896908.0,4294.53
3,2006,10 Jun 2006 - 18:00,Group B,"FIFA World Cup Stadium, Dortmund",Dortmund,Trinidad and Tobago,0,0,Sweden,,62959.0,0,0,MAIDIN Shamsul (SIN),PERMPANICH Prachya (THA),GHULOUM Eisa (UAE),97410100,97410004,TRI,SWE,D,47,16,25.75,2200.0,20300.0,1372598.0,14789.47,2.10,624.0,538575.0,9982709.0,53950.79
4,2006,10 Jun 2006 - 21:00,Group C,"FIFA World Cup Stadium, Hamburg",Hamburg,Argentina,2,1,Ivory Coast,,49480.0,2,0,DE BLEECKERE Frank (BEL),HERMANS Peter (BEL),VROMANS Walter (BEL),97410100,97410005,ARG,CIV,W,9,32,14.80,591.0,637717.0,44688864.0,14270.15,26.35,1348.0,40360.0,24905843.0,1620.50


In [20]:
test_df = pd.read_csv('results2018.csv')
test_df = test_df.replace({'IR Iran': 'Iran'})
test_df = test_df.rename(
    columns={'Home Result': 'Home Team Result', 'Team 1 Score': 'Home Team Goals', 'Team 2 Score': 'Away Team Goals'}
)

test_ranks = rank_game(test_df, wc2018_rankings)
test_ranks = country_game(country_factors, test_ranks)

test_ranks['Year'] = [2018 for i in range(len(test_ranks))]

In [21]:
locations = {
    2018: 'Russia',
    2014: 'Brazil',
    2010: 'Russia',
    2006: 'Germany'
}
    

In [22]:
temp_list_train = []
rain_list_train = []

temp_list_test = []
rain_list_test = []


for i in range(len(merged_country_info)):
    location = locations[merged_country_info['Year'][i]]
    
    temp_list_train.append(list(country_factors[country_factors.country == location].temp_in_celcius)[0])
    rain_list_train.append(list(country_factors[country_factors.country == location].rain_mm_per_year)[0])

for i in range(len(test_ranks)):
    location = locations[test_ranks['Year'][i]]
    
    temp_list_test.append(list(country_factors[country_factors.country == location].temp_in_celcius)[0])
    rain_list_test.append(list(country_factors[country_factors.country == location].rain_mm_per_year)[0])

merged_country_info['Host Temp'] = temp_list_train
merged_country_info['Host Rain'] = rain_list_train
    
test_ranks['Host Temp'] = temp_list_test
test_ranks['Host Rain'] = rain_list_test


In [23]:
merged_country_info['Home Temp Diff'] = merged_country_info['Home Temp'] - merged_country_info['Host Temp']
merged_country_info['Home Rain Diff'] = merged_country_info['Home Rain'] - merged_country_info['Host Rain']
merged_country_info['Away Temp Diff'] = merged_country_info['Away Temp'] - merged_country_info['Host Temp']
merged_country_info['Away Rain Diff'] = merged_country_info['Away Rain'] - merged_country_info['Host Rain']


test_ranks['Home Temp Diff'] = test_ranks['Home Temp'] - test_ranks['Host Temp']
test_ranks['Home Rain Diff'] = test_ranks['Home Rain'] - test_ranks['Host Rain']
test_ranks['Away Temp Diff'] = test_ranks['Away Temp'] - test_ranks['Host Temp']
test_ranks['Away Rain Diff'] = test_ranks['Away Rain'] - test_ranks['Host Rain']

In [24]:
country_ids = {}
for i in range(len(merged_country_info)):
    home_name, home_id = merged_country_info['Home Team Name'][i], merged_country_info['Home Team Initials'][i]
    away_name, away_id =merged_country_info['Away Team Name'][i], merged_country_info['Away Team Initials'][i]
    
    if not country_ids.get(home_name):
        country_ids[home_name] = home_id
    if not country_ids.get(away_name):
        country_ids[away_name] = away_id
for i in range(len(test_df)):
    home_name, home_id = test_df['Home Team Name'][i], test_df['Home Team Initials'][i]
    away_name, away_id =test_df['Away Team Name'][i], test_df['Away Team Initials'][i]
    
    if not country_ids.get(home_name):
        country_ids[home_name] = home_id
    if not country_ids.get(away_name):
        country_ids[away_name] = away_id


In [25]:
betting_data = pd.read_csv('betting_data/odds_full.csv', index_col=0)

In [26]:
# cleaning
betting_data.team = betting_data.team.replace({
    'Iran 30000': 'Iran',
    'Trinidad & Tobago': 'Trinidad and Tobago',
    'North Korea': 'Korea DPR',
    'Korea': 'Korea Republic',
    'South Korea': 'Korea Republic',
    'Bosnia & Herzegovina': 'Bosnia and Herzegovina',
    'Serbia & Montenegro': 'Serbia and Montenegro',
})

# remove older cups and test data
betting_data = betting_data[betting_data.year.isin(range(2006, 2019))]

# add country initals
team_ids = []
for i in range(len(betting_data)):
    team_ids.append(country_ids[list(betting_data.team)[i]])
betting_data['team_id'] = team_ids

betting_data = betting_data.drop(columns=['team'])

In [27]:
betting_data = betting_data.rename(columns={'team_id': 'Home Team Initials', 'year': 'Year', 'final_line': 'Home Team Line'})

In [28]:
temp_train = merged_country_info.merge(betting_data, how='inner', on=['Home Team Initials', 'Year'])
temp_test = test_ranks.merge(betting_data, how='inner', on=['Home Team Initials', 'Year'])

In [29]:
betting_data = betting_data.rename(columns={'Home Team Initials': 'Away Team Initials', 'Home Team Line': 'Away Team Line'})

In [30]:
full_train = temp_train.merge(betting_data, how='inner', on=['Away Team Initials', 'Year'])
full_test = temp_test.merge(betting_data, how='inner', on=['Away Team Initials', 'Year'])

full_test.head()

,Group,Home Team Name,Away Team Name,Home Team Goals,Away Team Goals,Home Team Result,Home Team Initials,Away Team Initials,Home Rank,Away Rank,Home Temp,Home Rain,Home GDP (millions),Home Population,Home GDP per Person (dollars),Away Temp,Away Rain,Away GDP (millions),Away Population,Away GDP per Person (dollars),Year,Host Temp,Host Rain,Home Temp Diff,Home Rain Diff,Away Temp Diff,Away Rain Diff,Home Team Line,Away Team Line
0,A,Russia,Saudi Arabia,5,0,W,RUS,KSA,66,67,15.10,460.0,1527469.0,143964709.0,10610.02,24.65,59.0,683827.0,33554343.0,20379.69,2018,15.1,460.0,0.00,0.0,9.55,-401.0,7500,150000
1,A,Uruguay,Saudi Arabia,1,0,W,URU,KSA,17,67,17.55,1265.0,58415.0,3469551.0,16836.47,24.65,59.0,683827.0,33554343.0,20379.69,2018,15.1,460.0,2.45,805.0,9.55,-401.0,2800,150000
2,A,Russia,Egypt,3,1,W,RUS,EGY,66,46,15.10,460.0,1527469.0,143964709.0,10610.02,22.10,51.0,237073.0,99375741.0,2385.62,2018,15.1,460.0,0.00,0.0,7.00,-409.0,7500,32500
3,A,Saudi Arabia,Egypt,2,1,W,KSA,EGY,67,46,24.65,59.0,683827.0,33554343.0,20379.69,22.10,51.0,237073.0,99375741.0,2385.62,2018,15.1,460.0,9.55,-401.0,7.00,-409.0,150000,32500
4,Quarter-finals,Russia,Croatia,2,2,L,RUS,CRO,66,18,15.10,460.0,1527469.0,143964709.0,10610.02,10.90,1113.0,54516.0,4164783.0,13089.76,2018,15.1,460.0,0.00,0.0,-4.20,653.0,7500,3500


In [31]:
columns_keep = [
    'Home Rank', 'Away Rank', 'Home Team Line', 'Away Team Line', 
    'Home GDP per Person (dollars)', 'Home GDP (millions)', 'Home Population', 'Home Temp Diff', 'Home Rain Diff',  
    'Away GDP per Person (dollars)', 'Away GDP (millions)', 'Away Population', 'Away Temp Diff', 'Away Rain Diff', 
]

In [32]:
full_train_X = full_train[columns_keep]
full_train_y = full_train[['Home Team Result']]

full_test_X = full_test[columns_keep]
full_test_y = full_test[['Home Team Result']]

In [33]:
full_train.to_csv('train_data.csv')

In [34]:
full_test.to_csv('test_data.csv')